In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Lab 7: Rank and Singular Value Decomposition

We have previously encountered the notion of **singular** matrices. For a singular matrix $A$, we've learned that:

1. $A^{-1}$ does not exist.
2. The equation $Ax = b$ does not have a unique solution.
3. The condition number $\text{cond}(A)$ is infinite.
4. The determinant $\text{det}(A)$ equals $0$.

**Singular Value Decomposition** (SVD) provides a comprehensive framework that ties these concepts together. Before delving into SVD, let's recap the concepts of **rank** and <b>outer product</b>.

### Rank

The **rank** of a matrix $A$ has several definitions, but for our purposes, we'll adopt the following:

> The **rank** of a matrix $A$ is the number of linearly independent rows (or columns) it possesses.

**Example** - Consider the matrix with three rows $r_1$, $r_2$, and $r_3$ (dashed lines represent the rest of the row):
$$
A = \begin{bmatrix} -- & r_1 & -- \\ 
                    -- & r_2 & -- \\
                    -- & r_3 & -- \end{bmatrix} =
    \begin{bmatrix} 1 & 1 & 0 \\ 
                    0 & 0 & 1 \\
                    2 & 2 & 0 \end{bmatrix}.
$$
1. We observe two linearly independent rows ($r_1$ and $r_2$). Thus, $\text{rank}(A) = 2$.
2. $r_3$ is not linearly independent, as it can be expressed as a linear combination of $r_1$ and $r_2$ (specifically, $r_3 = 2*r_1 + 0*r_2$).

In [2]:
A = np.array( [[1, 1, 0],
               [0, 0, 1],
               [2, 2, 0]] )
np.linalg.matrix_rank(A)

2

### Outer Product
For a given row vectors $u$ and $v$ of shape $(m, 1)$. 
>1. Outer product: $u v^{T}: (m, 1)\cdot(1, m) = (m, m)$ dimensional matrix. 
>2. Inner product: $u^{T} v: (1, m)\cdot(m, 1) = 1$ dimensional scalar. Also called dot product.

In [3]:
a = np.random.rand(5, 1)
b = np.random.rand(5, 1)
outer_prod = a @ b.T
inner_prod = b.T @ a 

#Note: if a is a 1D ndarray, then you have to use np.outer as you as you cannot transpose 
a     = np.random.rand(5)
b     = np.random.rand(5)
outer = np.outer(a, b)
outer = np.inner(a, b)

### Singular Value Decomposition (SVD)

The Singular Value Decomposition (SVD) is a mathematical technique that breaks down a matrix \( A \) into three component matrices, each providing valuable insights into the original matrix's structure and characteristics.

>$$ A = U \Sigma V^T $$
>
>1. $U$ is an $m \times m$ orthogonal matrix.
>2. $\Sigma$ is an $m \times n$ diagonal matrix containing the singular values ($\sigma_1 > \sigma_2 > \dots > \sigma_n \geq 0$).
>3. $V$ is an $n \times n$ orthogonal matrix.

In [4]:
# Generate a random full rank matrix 
A = np.random.rand(10, 10)
m, n = A.shape

# compute SVD of A
U, S, V = np.linalg.svd(A)

# reconstruct A
A_ = U @ np.diag(S) @ V

#verify properties 
assert np.isclose(A, A_).all() 
assert np.isclose(U@np.linalg.inv(U), np.eye(m)).all()
assert np.isclose(np.linalg.inv(V)@V, np.eye(n)).all()


**Geometrical Interpretation of SVD:**

In geometric terms, matrix $A$ <i>rotates</i> and <i>scales</i> the vector $b$ in space.

1. The singular vectors in matrices $U$ and $V$ represent **rotational** operations on the vector $b$.
2. The corresponding singular values $\sigma_i$ represent **scaling** operations on the vector $b$.

**Relating Condition Number to SVD:**

Since the singular values represent the action of $scaling$ by the matrix $A$, we can compute the condition as follows- 
$$
\kappa(A) = \frac{\text{max scaling}}{\text{min scaling}}=\frac{\max \frac{||Ax||}{||x||}}{\min \frac{||Ax||}{||x||}}=\frac{\sigma_1}{\sigma_n}.  (\text{Since $\sigma_1 > \sigma_2 \cdots \sigma_n \geq0$})
$$
>*Note:* If the matrix is singular, then $\sigma_n = 0$, which  implies $\kappa(A)=\frac{\sigma_1}{\sigma_n}=\infty$.

In [5]:
A = np.random.rand(10, 10)
U, S, V = np.linalg.svd(A) 

cond_num_numpy = np.linalg.cond(A) 
cond_num_svd   = max(S)/min(S)

assert np.isclose(cond_num_svd, cond_num_numpy)

**Relating SVD to Frobenius norm**

The Frobenius norm of a matrix is defind as -

$$
\|A\|_F = \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n} |a_{ij}|^2}
$$


If we plug in in the SVD decomposition and simplify we get


$$
\|A\|_F = \sqrt{\sum_{i=1}^{r} \sigma_i^2}
$$

Side note: Similar to vectors, we can compute a <i>residual</i> matrix to compare two matrices (say $A$ and $A'$) and compute the Frobenius norm as 

$$
||R||_F = \|A - A'\|_F.
$$

---

## Rank Reduction using Singular Value Decomposition (SVD)

The Singular Value Decomposition (SVD) breaks down a matrix $A$ into three components: the left singular vectors ($U$), the singular values ($\Sigma$), and the right singular vectors ($V^\text{T}$). These components are arranged as follows:

$$
A = U \Sigma V^\text{T} = 
\begin{bmatrix}
| & & | & |& & | \\
| & & | & |& & | \\
| & & | & |& & | \\
u_{1} & \dots & u_{r} & u_{r+1} & \dots & u_{m} \\
| & & | & |& & | \\
| & & | & |& & | \\
| & & | & |& & | \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{1} & & & & & \\
& \ddots & & & & \\
& & \sigma_{r} & & & \\
& & & 0 & & \\
& & & & \ddots & \\
& & & & & 0 \\
\end{bmatrix}
\begin{bmatrix}
\textemdash & \textemdash & v_{1}^{\text{T}} & \textemdash & \textemdash \\
& & \vdots & & \\
\textemdash & \textemdash & v_{r}^{\text{T}} & \textemdash & \textemdash \\
\textemdash & \textemdash & v_{r+1}^{\text{T}} & \textemdash & \textemdash \\
& & \vdots & & \\
\textemdash & \textemdash & v_{n}^{\text{T}} & \textemdash & \textemdash \\
\end{bmatrix}
$$

Here:
1. $r$ is the rank of the matrix $A$.
2. For non-singular square matrices, $\Sigma$ has non-zero singular values.
3. For singular or non-square matrices, $\Sigma$ has $r$ non-zero singular values along the main diagonal.

By simplifying the matrix product, $A$ we get:

$$ A = \sum_{i=1}^{r} \sigma_i u_i v_i^{\text{T}} $$

If we truncate this sum to include only the first $m$ terms, the resulting matrix $A'$ has a rank of $m$, reducing the complexity of $A$:

$$ A' = \sum_{i=1}^{m} \sigma_i u_i v_i^{\text{T}} $$

By selecting $m < \text{rank}(A)$, we retain the most significant components of $A$, effectively reducing its rank and complexity. This process is known as rank reduction using SVD.

In [6]:
# Rank reduction (change r value to reduce it to a rank you want) using the loop summation
A_approx = np.zeros(shape=A.shape)
r = 9
for i in range(r):
    A_approx += S[i] * np.outer(U[:, i], V[i, :])
    
print(f"L2 error for rank 6 reduction: {np.linalg.norm(A-A_approx)}")

L2 error for rank 6 reduction: 0.012853178678765134


> **Note:** We can also represent the truncation in the matrix product form using array slicing, 

$$ A' = U[:, :r] \, \Sigma[:r, :r] \, V^{T}[:r, :] $$

In [7]:
# You can also do the above summation operation using slicing (we will be using this to do the next experiment for brevity)
A_approx = U[:, :r] @ np.diag(S[:r]) @ V[:r, :]

print(f"L2 error for rank 6 reduction: {np.linalg.norm(A-A_approx)}")

L2 error for rank 6 reduction: 0.012853178678765299


## Rank Reduction Example: Low rank approximations of images

An image is represented as a 3D matrix, with each element denoting a pixel's RGB color intensity (ranging from 0 to 255). To set a pixel's intensity, we use the notation: $$X[i, j, :] = [r, g, b]$$

In this demonstration, images are converted to grayscale, resulting in a 2D matrix where each element represents the average of the RGB values: $$X_{\text{grey}}[i, j] = \frac{r + g + b}{3}$$

In [8]:
X = plt.imread("henley.jpg").convert('L')
X = X.mean(axis=2) # take the mean of the rgb intensities (the third dimension)
#X = 0.299*X[:, :, 0] + 0.587*X[:, :, 1] + 0.114*X[:, :, 2]

plt.title(f'Full Rank :{np.linalg.matrix_rank(X)}')
plt.imshow(X)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'henley.jpg'

In [ ]:
U, S, V  = np.linalg.svd(X, full_matrices=False)
X_approx = np.zeros(shape=X.shape) 

# do rank reductions for the following ranks 
ranks     = [5, 10, 200, 700]
#ranks    = np.arange(1, 500)

# perform rank reductions for ranks defined above and compute reconstruction error
X_lowrank = []
errors    = []
for r in ranks:
    X_approx = S[:r] * U[:, :r] @ V[:r, :] # (you can also use the loop version (see cell #5)
    errors.append(np.linalg.norm(X_approx - X)/np.linalg.norm(X))
    X_lowrank.append(X_approx)

#plot values
plt.plot(ranks, errors, '-o')
plt.xlabel("reduced rank $r$")
plt.ylabel("Reconstruction error $||A - U \Sigma V^{T}||_2 / ||A||_2$")
plt.show()

In [ ]:
U, S, V  = np.linalg.svd(X, full_matrices=False)
X_approx = np.zeros(shape=X.shape) 

# do rank reductions for the following ranks 
ranks     = [5, 10, 200, 700]
#ranks    = np.arange(1, 500)

# perform rank reductions for ranks defined above and compute reconstruction error
X_lowrank = []
errors    = []
for r in ranks:
    X_approx = S[:r] * U[:, :r] @ V[:r, :] # (you can also use the loop version (see cell #5)
    errors.append(np.linalg.norm(X_approx - X)/np.linalg.norm(X))
    X_lowrank.append(X_approx)

#plot values
plt.plot(ranks, errors, '-o')
plt.xlabel("reduced rank $r$")
plt.ylabel("Reconstruction error $\frac{||A - U \Sigma V^{T}||_2}{||A||_2}$")
plt.show()

In [ ]:
# compute error and display approximations
fig, axs = plt.subplots(1, len(ranks), figsize=(10, 10))

for i in range(len(ranks)):    
    axs[i].set_title(f"rank: {ranks[i]} | err: {errors[i]:.3f}")
    axs[i].imshow(X_lowrank[i], cmap='gray')
    
plt.tight_layout()
plt.show()

In [ ]:

X = plt.imread("halfdome.jpg")
X.shape

In [ ]:
U0, S0, V0  = np.linalg.svd(X0, full_matrices=False)
U1, S1, V1  = np.linalg.svd(X1, full_matrices=False)
U2, S2, V2  = np.linalg.svd(X2, full_matrices=False)

In [ ]:
X = X0
for r in ranks:
    X_approx = S0[:r] * U0[:, :r] @ V0[:r, :] # (you can also use the loop version (see cell #5)
    errors.append(np.linalg.norm(X_approx - X)/np.linalg.norm(X))
    X_lowrank.append(X_approx)
errors

In [ ]:
X = X1
for r in ranks:
    X_approx = S1[:r] * U1[:, :r] @ V1[:r, :] # (you can also use the loop version (see cell #5)
    errors.append(np.linalg.norm(X_approx - X)/np.linalg.norm(X))
    X_lowrank.append(X_approx)
errors

In [ ]:
X = X2
for r in ranks:
    X_approx = S2[:r] * U2[:, :r] @ V2[:r, :] # (you can also use the loop version (see cell #5)
    errors.append(np.linalg.norm(X_approx - X)/np.linalg.norm(X))
    X_lowrank.append(X_approx)
errors

In [ ]:
for r in ranks:
    X_approx = S[:r] * U[:, :r] @ V[:r, :] # (you can also use the loop version (see cell #5)
    errors.append(np.linalg.norm(X_approx - X)/np.linalg.norm(X))
    X_lowrank.append(X_approx)
errors

In [ ]:
plt.imshow(X[:, :, 2])